<a href="https://colab.research.google.com/github/fsternini/brats-task2/blob/development/brats_task2_launcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First, verification of Python version is completed
!which python # should return /usr/local/bin/python
!python --version

/usr/local/bin/python
Python 3.10.12


In [2]:
# Google Drive is mounted, to reach all relevant data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/fsternini/brats-task2.git
%cd 'brats-task2'
# Switch to the most updated branch
!git switch development
# Collect the most updated version of the repo
!git pull

Cloning into 'brats-task2'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 116 (delta 47), reused 73 (delta 20), pack-reused 0
Receiving objects: 100% (116/116), 89.17 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/brats-task2
Branch 'development' set up to track remote branch 'development' from 'origin'.
Switched to a new branch 'development'
Already up to date.


In [4]:
# Install all requirements for the code
!pip install -r requirements.txt
#!pip install pydicom numpy torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('code')
import common.common

In [ ]:
#from common.common import DatasetCreator

In [ ]:
#dataset_creator = DatasetCreator('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset')

In [ ]:
from common.common import Dataset
ds = Dataset('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/datasetdataset_structure.json')

In [ ]:
from common.common import Volume

In [ ]:
vol = Volume('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00000/FLAIR')
vol.load_dicom_images()

In [ ]:
vol.show_volume(0)

interactive(children=(IntSlider(value=0, description='slice_idx', max=399), Output()), _dom_classes=('widget-i…

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train_labels.csv')

In [ ]:
df

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1
...,...,...
580,1005,1
581,1007,1
582,1008,1
583,1009,0


In [9]:
try:
  data_dict = json.load('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/data.json')
except:
  data_dict = {}

In [75]:
import os
for i in df["BraTS21ID"]:
  if i not in data_dict:
    data_dict[i] = {"class": df.loc[df["BraTS21ID"] == i]["MGMT_value"].item()}
    dir_path = f"/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/{i:05d}"
    data_dict[i]["data"]={}
    for dirname in os.listdir(dir_path):
      data_dict[i]["data"][dirname] = {
          "folder":f"{dir_path}/{dirname}",
          "orientation": orientation(f"{dir_path}/{dirname}")
      }

KeyboardInterrupt: 

In [11]:
dir_path=f"/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/{i:05d}"

In [56]:
p = data_dict[5]["data"]["FLAIR"]

In [76]:
data_dict

{0: {'class': 1,
  'data': {'FLAIR': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00000/FLAIR',
    'series': 1},
   'T1w': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00000/T1w',
    'series': 1},
   'T1wCE': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00000/T1wCE',
    'series': 1},
   'T2w': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00000/T2w',
    'series': 1}}},
 2: {'class': 1,
  'data': {'FLAIR': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00002/FLAIR',
    'series': 1},
   'T1w': {'folder': '/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00002/T1w',
    'series': 1},
   'T1wCE': 

In [37]:
import pydicom

dmc_prova = pydicom.dcmread(p+'/Image-82.dcm')


In [72]:
def series_count(folder):
  s = set()
  p = folder
  for img in os.listdir(p):
    dcm = pydicom.dcmread(p+'/'+img)
    if dcm.SeriesInstanceUID not in s:
      s.add(dcm.SeriesInstanceUID)
  return len(s)
def orientation(folder):
  dcm = pydicom.dcmread(folder+'/'+os.listdir(folder)[0])

  return dcm.ImageOrientationPatient

In [74]:
dmc_prova


Dataset.file_meta -------------------------------
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['DERIVED', 'SECONDARY']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.8.498.33585239833757386776301770865782817633
(0008, 0050) Accession Number                    SH: '00000'
(0008, 0060) Modality                            CS: 'MR'
(0008, 103e) Series Description                  LO: 'FLAIR'
(0010, 0010) Patient's Name                      PN: '00000'
(0010, 0020) Patient ID                          LO: '00000'
(0018, 0023) MR Acquisition Type                 CS: '3D'
(0018, 0050) Slice Thickness                     DS: '1.2'
(0018, 0081) Echo Time                           DS: None
(0018, 0082)

In [73]:
orientation('/content/drive/MyDrive/Data Science/rsna-miccai-brain-tumor-radiogenomic-classification/dataset/train/00100/FLAIR')

[1, 0, 0, 0, 1, 0]